In [ ]:
import os
from models.SPIDER import SPIDER
from dataset.SPIDER_dataset import SPIDERDataset
from utils import get_cell_expression_data, to_entrez_id, min_max_normalize, to_symbol

In [ ]:
interaction_methods_path = '' # should have the columns of g1, g2 (the interaction genes)
# all data should be of type dataframe with the entrez-ids as index
co_locations_path = ''
co_abundance_path = ''
h_sapiens_path = ''
gene_expression_path = ''
proteins_expression_path = ''
locations_path = ''
interaction_methods = pd.read_csv(interaction_methods_path)
interaction_methods_dict = dict(zip(zip(interaction_methods.g1, interaction_methods.g2),
              [t[3:] for t in interaction_methods.itertuples()]))
location_data = pd.read_csv(locations_path)
co_locations_data = pd.read_csv(co_locations_path)
co_abundance = pd.read_csv(co_abundance_path)
proteins_expressions = pd.read_csv(proteins_expression_path)
expression_data = pd.read_csv(gene_expression_path)
h_sapiens = pd.read_csv(h_sapiens_path)

expression_len = 2
prots_len = 3
locations_len = 2 * location_data.shape[1] + 1
methods_len = interaction_methods.shape[1] - 2

In [ ]:
human_data_dir = 'human_dataset
train_interactions = [] # devide the interaction set into three groups
val_interactions = []
test_interactions = []
for interactions, dataset_name in zip([train_interactions, val_interactions, test_interactions], ['train', 'val', 'test']):
  data, labels, edges = get_data_matrix(
    interactions, ground_truth, gene_expression=expression_data,
    prot_expression=pd.DataFrame(min_max_normalize(proteins_expressions)),
    co_abundance=co_abundance,
    locations=location_data, methods=interaction_methods_dict, co_locations=co_locations_data)
  data['edges'] = edges
  pd.DataFrame(data).to_csv(os.path.join(human_data_dir, f'{dataset_name}_X.csv'), index=False)
  pd.DataFrame(labels).to_csv(os.path.join(human_data_dir, f'{dataset_name}_y.csv'), index=False)

In [ ]:

datasets = {
    x: SPIDERDataset(data=d, y_data=y, expression_size=expression_len, locations_size=locations_len,
                  prot_size=prots_len, methods_size=methods_len) for x, d, y in
    zip(['train', 'val', 'test'], [X_train, X_val, X_test], [y_train, y_val, y_test])}

torch.manual_seed(seed=1234)
np.random.seed(1234)
model = SPIDER(expression_size=expression_len, locations_size=locations_len, prot_size=prots_len, graph_matrix=datasets['train'].graph_matrix,
                  second_input_size=datasets['train'].interaction.shape[-1],
                  hidden_size=64, p=0.3)
model.train_all(datasets, epochs=750, learning_rate=1e-3)
